## The Functions of the Discrete Voter Model

This notebook implements the following $6$ subroutines for the **discrete voter model**:
1. `make_grid`: create a probabilistic grid: an $\mathbf{R}^n$ array of values, $\omega$, that all sum to $1$
2. `shift_weight`: shift weight in a grid in a reversible way
3. `expec_votes`: given a grid, a candidate, and a district description, output the expectation of votes that candidate received
4. `prob_votes`: given a grid, a candidate, a district description, and the observed vote outcomes, output the probability of seeing that outcome
5. `mcmc`: run a Markov Chain Monte Carlo method on a state space of grids
6. `hill_climb`: optimize the expectation or probability with gradient descent

### `make_grid`
Create a probabilistic grid: an $\mathbf{R}^n$ array of values, $\omega$, that all sum to $1$.

In [77]:
import numpy as np
from operator import mul
import functools

In [2]:
def make_grid(num_groups, matrix_size, random=True):
    """
    Create a probabilistic grid.
    
    num_groups (int): the number of groups to be represented 
    matrix_size (int): the dimensions of the matrix
    random (bool): whether the grid should be initialized uniformly
    or randomly
    
    return: a probabilistic grid 
    """
    if random:
        matrix = np.random.rand(num_groups - 1, matrix_size, matrix_size)
    else:
        matrix = np.ones((num_groups - 1, matrix_size, matrix_size))
    return matrix / matrix.sum()

### `shift_weight`
Shift weight in a grid in a reversible way.

In [3]:
def shift_weight(grid, shift_type="uniform"):
    """
    Shift the weight in a probabilistic grid.
    
    grid (NumPy array): the probabilistic grid to be perturbed
    shift_type (string): the type of shift to be done. One of:
        1. uniform (default): add a uniform random variable to each cell then re-normalize
        2. shuffle: shuffle the matrix (O(n))
        3. right: shift 10% of weight to the right
        4. left: shift 10% of weight to the left
    
    return: a probabilistic grid
    """
    if shift_type == "shuffle":
        np.random.shuffle(grid)
        return grid
    elif shift_type == "right":
        rolled = np.roll(grid, 1, axis=1)
        return grid + (rolled * 0.1) - (grid * 0.1)
    elif shift_type == "left":
        rolled = np.roll(grid, -1, axis=1)
        return grid + (rolled * 0.1) - (grid * 0.1)
    else:
        new_grid = grid + np.random.uniform()
        return new_grid / new_grid.sum()

### `expec_votes`
Given a grid, a candidate, and a district description, output the expectation of votes that candidate received.

In [156]:
def expec_votes(grid, demo):
    """
    Find the expectation of votes that a candidate
    received in a given election, with a given
    probabilistic grid.
    
    grid (NumPy array): the probabilistic grid for the precinct
    and candidate
    demo (dict): the demographics of the district
    """
    # Randomly select a cell based on their probabilities
    flat_index = np.random.choice(range(grid.size), p=test.flatten())
    index = np.unravel_index(flat_index, grid.shape)
    
    # Calculate the vote outcomes given the cell selected
    
    vote_outcome = np.zeros(len(demo))
    
    for num, group in enumerate(demo):
        # Find the probabilities the cell represents for each group
        pct = index[1] / grid.shape[1]
        vote_outcome[num] += demo[group] * pct
        
    return np.sum(vote_outcome)

### `prob_votes`
Given a grid, a candidate, a district description, and the observed vote outcomes, output the probability of seeing that outcome.

In [ ]:
def prob_votes(grid, demo, observed):
    """
    Find the expectation of votes that a candidate
    received in a given election, with a given
    probabilistic grid.
    
    grid (NumPy array): the probabilistic grid for the precinct
    and candidate
    demo (dict): the demographics of the district
    observed (int): the observed number of votes the candidate received
    """
    # Randomly select a cell based on their probabilities
    flat_index = np.random.choice(range(grid.size), p=test.flatten())
    index = np.unravel_index(flat_index, grid.shape)
    
    # Calculate the vote outcomes given the cell selected
    
    vote_outcome = np.zeros(len(demo))
    
    for num, group in enumerate(demo):
        # Find the probabilities the cell represents for each group
        pct = index[1] / grid.shape[1]
        vote_outcome[num] += demo[group] * pct
        
    return np.sum(vote_outcome)

In [157]:
matrix_dim = test.shape[1:]

In [158]:
sum([expec_votes(test, demo) for i in range(1000)]) / 1000

8.09025

In [92]:
demo = {"Black": 10, "white": 8, "Latinx": 5}

In [94]:
list(enumerate(demo))

[(0, 'Black'), (1, 'white'), (2, 'Latinx')]

In [ ]:
expec_votes(test, {""})

In [89]:
sum([1,2,3])

6

In [84]:
functools.reduce(mul, matrix_dim)

16

In [87]:
51 % 15

6

In [68]:
test = make_grid(2, 4)

In [69]:
test2.shape

(2, 4, 4)

In [70]:
range(len(test.flatten()))

range(0, 16)

In [71]:
test.size

16

In [76]:
test.shape[1:]

(4, 4)

In [10]:
np.random.choice(test.flatten())

0.055326931188884355

In [11]:
np.random.choice(range(test.size), p=test.flatten())

7

In [74]:
np.unravel_index(15, test.shape)

(0, 3, 3)

In [23]:
test[index]

0.02770230322508814

In [64]:
index

(0, 3, 3)

In [66]:
test.shape

(1, 4, 4)

In [59]:
test2 = make_grid(3, 4)

In [60]:
test2

array([[[0.04356955, 0.01996903, 0.03762756, 0.05104305],
        [0.03662927, 0.03660784, 0.02906147, 0.00742786],
        [0.00840191, 0.04983294, 0.02012603, 0.04973958],
        [0.04368894, 0.036408  , 0.03341903, 0.01210207]],

       [[0.02859783, 0.03602456, 0.0509068 , 0.04943169],
        [0.04433833, 0.02533709, 0.04677605, 0.0040374 ],
        [0.00580014, 0.03857154, 0.01168324, 0.02933397],
        [0.02763215, 0.00702472, 0.02437048, 0.0544799 ]]])

In [63]:
test2.shape

(2, 4, 4)

In [52]:
np.unravel_index(3524, test2.shape)

(1, 20, 24)

In [51]:
test2.size

122500